In [2]:
import os
import json
import re
import spacy
from src.utils import utils
from tqdm.notebook import tqdm
from src.utils.transl import text_to_target_lang
import spacy
from spacy.cli.download import download
from transformers import pipeline  # Add this import statement
def extract_named_entities(ner_model, text):
    """
    Extract named entities from text.

    Parameters
    ----------
    ner_model : LANGUAGE
        NAMED ENTITY REGOGNITION MODEL.
    text : STR
        TEXT TO EXTRACT NAMED ENTITIES FROM.

    Returns
    -------
    entities : LIST
        LIST OF NAMED ENTITIES.

    """
    nlp = ner_model(text)

    entities = {}
    for ent in nlp.ents:
        if ent.label_ not in entities:
            entities[ent.label_] = [ent.text]
        else:
            entities[ent.label_].append(ent.text)
    return entities

def clean_provenance(prov):
    """
    Proprocess provenance text.

    Parameters
    ----------
    prov : STR
        PROPVENANCE TEXT.

    Returns
    -------
    prov : STR
        PREPROCESSED PROPVENANCE TEXT.

    """ 
    # Remove the parts of the provenance that are inside brackets (to facilitate extracting events)
    prov = re.sub("\s[\[\(].*?[\)\]]", "", prov)
    
    # This is to avoid the splitting of provenance with such kind of dots 
    prov = re.sub("Dr\.|Mr\.|Ms\.|Mrs\.|Prof\.|St\.|Rev\.|acc\.|vol\.|no\.|pl\.|inv\.", "", prov, flags=re.IGNORECASE)
    
    # The ":" and ";" act as event separator such as the "."
    prov = prov.replace(':', '.')
    prov = prov.replace(';', '.')
    
    # Cleaning
    prov = prov.replace('. ,', ',')
    prov = prov.replace('. ;', ',')    
    prov = prov.replace('.,', ',')
    
    prov = prov.replace('_', '')
    prov = prov.replace('<em>', '')


    
    return prov


def samply(population, desired_power=0.95, alpha=0.05, effect_size=0.2, ratio=0.5, alternative='two-sided'):
    import statsmodels.api as sm
    import numpy as np
    """
    Estimate the required sample size for a given population size and desired statistical parameters.

    Parameters:
    population (list): Your dataset
    desired_power (float): Desired statistical power (e.g., 0.95 for 95% power).
    alpha (float): Significance level (e.g., 0.05 for 5% significance).
    effect_size (float): Effect size (Cohen's d, small effect).
    ratio (float): Ratio of sample size to population size (adjust based on dataset size).
    alternative (str): Type of hypothesis test ('two-sided', 'larger', or 'smaller').

    Returns:
    int: The estimated required sample size.
    """
    # Calculate the required sample size
    population_size = int(len(population))
    required_sample_size = sm.stats.tt_ind_solve_power(
        effect_size=effect_size, # Effect size (Cohen's d, small effect)
        alpha=alpha, # Significance level (e.g., 0.05 for 5% significance)
        power=desired_power, # Desired power (e.g., 80% power)
        ratio=ratio, # You can adjust this ratio based on your dataset size
        alternative=alternative  # Change to 'larger' or 'smaller' based on your hypothesis
    )

    # Calculate the percentage of the population that the required sample represents
    sample_percentage = (required_sample_size / population_size) * 100
    print(f"Desired Power: {desired_power}")
    print(f"Significance Level (alpha): {alpha}")
    print(f"Effect Size (Cohen's d): {effect_size}")
    print(f"Total Population Size: {population_size}")
    print(f"Required Sample Size for Desired Power: {required_sample_size:.0f}")
    print(f"Sample Size as a Percentage of the Population: {sample_percentage:.2f}%")
    required_sample_size = int(f'{required_sample_size:.0f}')
    sample = required_sample_size
    
    return sample

def extract_named_entities(text, spacy_model, old_model, nlp):
    """
    Extract named entities from text.
    
    Parameters
    ----------
    text : STR
        TEXT TO EXTRACT NAMED ENTITIES FROM.

    Returns
    -------
    entities : LIST
        LIST OF NAMED ENTITIES.

    Detectable Languages:
    Modern: Bulgarian (bg), Croatian (hr), Czech (cs), Danish (da), Dutch (nl), English (en),
    Estonian (et), Finnish (fi), French (fr), German (de), Greek (el), Hungarian (hu), Irish (ga),
    Italian (it), Latvian (lv), Lithuanian (lt), Maltese (mt), Polish (pl), Portuguese (pt),
    Romanian (ro), Slovak (sk), Slovenian (sl), Spanish (es), Swedish (sv), Russian (ru),
    Turkish (tr), Basque (eu), Catalan (ca), Albanian (sq), Serbian (se), Ukrainian (uk),
    Norwegian (no), Arabic (ar), Chinese (zh), Hebrew (he)

    Medieval: Middle High German (mhd), Latin (la), Middle Low German (gml), Old French (fro),
    Old Church Slavonic (chu), Early New High German (fnhd), Ancient and Medieval Greek (grc)
    """
    # Language detection using the specified model
    
    if spacy_model != old_model:
        try:
            nlp = spacy.load(spacy_model)
            print(f"{spacy_model} loaded")
        except OSError:
            print(f"Download {spacy_model}")
            try:
                download(spacy_model)  # Download the Spacy model
                nlp = spacy.load(spacy_model)
            except:
                nlp = spacy.load('en_core_news_md')
    else:
        spacy_model = spacy_model
        print(f"mantaining {old_model}")

    # Use the Spacy model for named entity recognition
    entities = {}
    for ent in nlp(text).ents:
        for k,v in new_dict.items():
            for i in v:
                if ent.label_ == i:
                    ent.label_ = k
        if ent.label_ not in entities:
            entities[ent.label_] = [ent.text]
        else:
            entities[ent.label_].append(ent.text)
    old_model = spacy_model
    return entities, old_model, nlp


In [216]:
download('mk_core_news_sm')

✔ Download and installation successful
You can now load the package via spacy.load('mk_core_news_sm')


In [208]:


# Dictionary to map NER labels for each model to English labels
ner_label_mapping = {}

# Define label mappings for each model
label_mappings = {
    'da_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'fr_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'lt_core_news_sm': {'GPE': 'GPE', 'LOC': 'LOC', 'ORG': 'ORG', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT', 'TIME': 'TIME'},
    'en_core_web_md': {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE',
                       'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY', 'NORP': 'NORP',
                       'ORDINAL': 'ORDINAL', 'ORG': 'ORG', 'PERCENT': 'PERCENT', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT',
                       'QUANTITY': 'QUANTITY', 'TIME': 'TIME', 'WORK_OF_ART': 'WORK_OF_ART'},
    'ja_core_news_sm': {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE',
                        'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY', 'MOVEMENT': 'MOVEMENT',
                        'NORP': 'NORP', 'ORDINAL': 'ORDINAL', 'ORG': 'ORG', 'PERCENT': 'PERCENT', 'PERSON': 'PERSON',
                        'PET_NAME': 'PERSON', 'PHONE': 'PHONE', 'PRODUCT': 'PRODUCT', 'QUANTITY': 'QUANTITY', 'TIME': 'TIME',
                        'TITLE_AFFIX': 'TITLE_AFFIX', 'WORK_OF_ART': 'WORK_OF_ART'},
    'es_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'zh_core_web_sm': {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE',
                       'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY', 'NORP': 'NORP',
                       'ORDINAL': 'ORDINAL', 'ORG': 'ORG', 'PERCENT': 'PERCENT', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT',
                       'QUANTITY': 'QUANTITY', 'TIME': 'TIME', 'WORK_OF_ART': 'WORK_OF_ART'},
    'fi_core_news_sm': {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE',
                        'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY', 'NORP': 'NORP',
                        'ORDINAL': 'ORDINAL', 'ORG': 'ORG', 'PERCENT': 'PERCENT', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT',
                        'QUANTITY': 'QUANTITY', 'TIME': 'TIME', 'WORK_OF_ART': 'WORK_OF_ART'},
    'de_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'ca_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'nb_core_news_sm': {'DRV': 'ORG', 'EVT': 'EVENT', 'GPE_LOC': 'GPE', 'GPE_ORG': 'GPE', 'LOC': 'LOC', 'MISC': 'MISC',
                        'ORG': 'ORG', 'PER': 'PERSON', 'PROD': 'PRODUCT'},
    'it_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'nl_core_news_sm': {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE',
                        'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY', 'NORP': 'NORP',
                        'ORDINAL': 'ORDINAL', 'ORG': 'ORG', 'PERCENT': 'PERCENT', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT',
                        'QUANTITY': 'QUANTITY', 'TIME': 'TIME', 'WORK_OF_ART': 'WORK_OF_ART'},
    'sv_core_news_sm': {'EVN': 'EVENT', 'LOC': 'LOC', 'MSR': 'ORG', 'OBJ': 'ORG', 'ORG': 'ORG', 'PRS': 'PERSON', 'TME': 'TIME',
                        'WRK': 'WORK_OF_ART'},
    'pl_core_news_sm': {'date': 'DATE', 'geogName': 'LOC', 'orgName': 'ORG', 'persName': 'PERSON', 'placeName': 'LOC', 'time': 'TIME'},
    'ro_core_news_sm': {'DATETIME': 'DATE', 'EVENT': 'EVENT', 'FACILITY': 'FAC', 'GPE': 'GPE', 'LANGUAGE': 'LANGUAGE',
                        'LOC': 'LOC', 'MONEY': 'MONEY', 'NAT_REL_POL': 'NORP', 'NUMERIC_VALUE': 'CARDINAL',
                        'ORDINAL': 'ORDINAL', 'ORGANIZATION': 'ORG', 'PERIOD': 'DATE', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT',
                        'QUANTITY': 'QUANTITY', 'WORK_OF_ART': 'WORK_OF_ART'},
    'pt_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'sl_core_news_sm': {'DERIV_PER': 'PERSON', 'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'},
    'hr_core_news_sm': {'DERIV_PER': 'PERSON', 'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PER': 'PERSON'}
}

# Populate the dictionary with mappings
for model, label_mapping in label_mappings.items():
    ner_label_mapping[model] = label_mapping

new_dict = dict()
long = label_mappings['ja_core_news_sm']
for k,v in long.items():
    new_dict[v] = []

for k,v in label_mappings.items():
    for key,value in v.items():
        for j,z in new_dict.items():
            if value == j:
                z.append(key)
            else:
                pass
for j,z in new_dict.items():
    clean = list(set(z))
    new_dict[j] = clean

others = new_dict['TITLE_AFFIX']+new_dict['PHONE']
others.append('MISC')
for x in new_dict['TIME']:
    new_dict['DATE'].append(x)
for x in new_dict['LOC']:
    new_dict['GPE'].append(x)
    
new_dict['OTHER'] = others
new_dict.pop('TITLE_AFFIX')
new_dict.pop('PHONE')
new_dict.pop('TIME')
for k,v in new_dict.items():
    print(k,'->',v)


CARDINAL -> ['CARDINAL', 'NUMERIC_VALUE']
DATE -> ['date', 'DATETIME', 'DATE', 'PERIOD', 'TME', 'time', 'TIME']
EVENT -> ['EVT', 'EVN', 'EVENT']
FAC -> ['FAC', 'FACILITY']
GPE -> ['GPE', 'GPE_ORG', 'GPE_LOC', 'placeName', 'LOC', 'geogName']
LANGUAGE -> ['LANGUAGE']
LAW -> ['LAW']
LOC -> ['placeName', 'LOC', 'geogName']
MONEY -> ['MONEY']
MOVEMENT -> ['MOVEMENT']
NORP -> ['NAT_REL_POL', 'NORP']
ORDINAL -> ['ORDINAL']
ORG -> ['MSR', 'ORGANIZATION', 'orgName', 'DRV', 'OBJ', 'ORG']
PERCENT -> ['PERCENT']
PERSON -> ['PER', 'PERSON', 'PRS', 'persName', 'DERIV_PER', 'PET_NAME']
PRODUCT -> ['PRODUCT', 'PROD']
QUANTITY -> ['QUANTITY']
WORK_OF_ART -> ['WRK', 'WORK_OF_ART']
OTHER -> ['TITLE_AFFIX', 'PHONE', 'MISC']


In [97]:
with open('datasets/final_db.json','r',encoding='utf-8') as f:
    final = json.load(f)


KeyError: 'cae03c8d51435f87bbac86923f4dd9ad82bdca396c1b6f76e721717e8fa67c5767fda7a561f070c9d62aa27d52f763604cb15429551d0863bc3e57e2d40ccd14'

In [71]:
import json
from tqdm.notebook import tqdm
    
file = 'datasets/lang_db.json'
artwork_index = 0
NER_eval = []
nn = 0
old_model = 'en_core_web_md'
models_used = []

# Define 'nlp' before the loop
nlp = spacy.load('en_core_web_md')

spacy_models = {
    'ca': 'ca_core_news_sm',  # Catalan
    'da': 'da_core_news_sm',  # Danish
    'de': 'de_core_news_sm',  # German
    'el': 'el_core_news_sm',  # Greek
    'en': 'en_core_web_md',  # English
    'es': 'es_core_news_sm',  # Spanish
    'fi': 'fi_core_news_sm',  # Finnish
    'fr': 'fr_core_news_sm',  # French
    'hr': 'hr_core_news_sm',  # Croatian
    'it': 'it_core_news_sm',  # Italian
    'ja': 'ja_core_news_sm',  # Japanese
    'ko': 'ko_core_news_sm',  # Korean
    'lt': 'lt_core_news_sm',  # Lithuanian
    'mk': 'mk_core_news_sm',  # Macedonian
    'nb': 'nb_core_news_sm',  # Norwegian Bokmål
    'nl': 'nl_core_news_sm',  # Dutch
    'pl': 'pl_core_news_sm',  # Polish
    'pt': 'pt_core_news_sm',  # Portuguese
    'ro': 'ro_core_news_sm',  # Romanian
    'sl': 'sl_core_news_sm',  # Slovenian
    'sv': 'sv_core_news_sm',  # Swedish
    'ru': 'ru_core_news_sm',  # Russian
    'uk': 'uk_core_news_sm',  # Ukrainian
    'xx_ent_wiki': 'xx_ent_wiki_sm',  # Multi-language Entity Recognition
    'xx_sent_ud': 'xx_sent_ud_sm',  # Multi-language Sentence Segmentation
    'zh': 'zh_core_web_sm',  # Chinese
}


def extract_named_entities(text, spacy_model, old_model, nlp):
    """
    Extract named entities from text.
    
    Parameters
    ----------
    text : STR
        TEXT TO EXTRACT NAMED ENTITIES FROM.

    Returns
    -------
    entities : LIST
        LIST OF NAMED ENTITIES.

    Detectable Languages:
    Modern: Bulgarian (bg), Croatian (hr), Czech (cs), Danish (da), Dutch (nl), English (en),
    Estonian (et), Finnish (fi), French (fr), German (de), Greek (el), Hungarian (hu), Irish (ga),
    Italian (it), Latvian (lv), Lithuanian (lt), Maltese (mt), Polish (pl), Portuguese (pt),
    Romanian (ro), Slovak (sk), Slovenian (sl), Spanish (es), Swedish (sv), Russian (ru),
    Turkish (tr), Basque (eu), Catalan (ca), Albanian (sq), Serbian (se), Ukrainian (uk),
    Norwegian (no), Arabic (ar), Chinese (zh), Hebrew (he)

    Medieval: Middle High German (mhd), Latin (la), Middle Low German (gml), Old French (fro),
    Old Church Slavonic (chu), Early New High German (fnhd), Ancient and Medieval Greek (grc)
    """
    # Language detection using the specified model
    
    if spacy_model != old_model:
        try:
            nlp = spacy.load(spacy_model)
            print(f"{spacy_model} loaded")
        except OSError:
            print(f"Download {spacy_model}")
            try:
                download(spacy_model)  # Download the Spacy model
                nlp = spacy.load(spacy_model)
            except:
                nlp = spacy.load('en_core_news_md')
    else:
        spacy_model = spacy_model
        print(f"mantaining {old_model}")

    # Use the Spacy model for named entity recognition
    entities = {}
    for ent in nlp(text).ents:
        #for k,v in new_dict.items():
         #   for i in v:
          #      if ent.label_ == i:
           #         ent.label_ = k
        if ent.label_ not in entities:
            entities[ent.label_] = [ent.text]
        else:
            entities[ent.label_].append(ent.text)
    old_model = spacy_model
    return entities, old_model, nlp

file = 'datasets/lang_db.json'

spacy_model = old_model
with open(file, 'r', encoding='utf-8') as input_f:
    data = json.load(input_f)
    datum = dict()
    import random

    # Get all items as a list of key-value pairs
    items = list(data['lots'].items())

    # Specify the number of random items you want to select
    sample_size = samply(items)  # Adjust this as needed
    
    # Select a random sample
    random_sample = random.sample(items, sample_size)
    #random_sample = items
    for key, value in tqdm(random_sample):
        json_object = value
        prov = value['lotProvenance']
        events_of_artworks = []

        if prov:
            try:
                for k, v in prov.items():
                    if v is None:
                        pass
                    elif k == 'provenance_0':
                        pass
                    else:
                        provenance = clean_provenance(v['text'])
                        spacy_model = v['spacy_model']

                        entities, old_model, nlp = extract_named_entities(provenance, spacy_model, old_model, nlp)
                        provs = []

                        for i, entity_ls in enumerate(entities):
                            provs.append((i, entity_ls, entities[entity_ls], old_model))

                            if len(entities[entity_ls]) > 1:
                                for j in entities[entity_ls]:
                                    tup = (entity_ls, j,old_model)
                                    if tup in NER_eval:
                                        pass
                                    else:
                                        NER_eval.append(tup)

                            else:
                                tup = (entity_ls, entities[entity_ls][0],old_model)
                                if tup in NER_eval:
                                    pass
                                else:
                                    NER_eval.append(tup)
            except Exception as e:
                print(f"Error processing provenance: {str(e)}")
            nn += 1
            




Desired Power: 0.95
Significance Level (alpha): 0.05
Effect Size (Cohen's d): 0.2
Total Population Size: 30422
Required Sample Size for Desired Power: 976
Sample Size as a Percentage of the Population: 3.21%


  0%|          | 0/976 [00:00<?, ?it/s]

mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
zh_core_web_sm loaded
mantaining zh_core_web_sm
en_core_web_md loaded
mantaining en_core_web_md
zh_core_web_sm loaded
en_core_web_md loaded
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
de_core_news_sm loaded
en_core_web_md loaded
fr_core_news_sm loaded
mantaining fr_core_news_sm
en_core_web_md loaded
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
fr_core_news_sm loaded
en_core_web_md loaded
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
mantaining en_core_web_md
nl_core_news_sm loaded
en_core_web_md loaded
mantaining en_core_web_md
mantaining en_core_web_md
nl_core_news_sm loaded
en_cor

In [74]:

spacy_models = {
    'ca': 'ca_core_news_sm',  # Catalan
    'da': 'da_core_news_sm',  # Danish
    'de': 'de_core_news_sm',  # German
    'el': 'el_core_news_sm',  # Greek
    'en': 'en_core_web_md',  # English
    'es': 'es_core_news_sm',  # Spanish
    'fi': 'fi_core_news_sm',  # Finnish
    'fr': 'fr_core_news_sm',  # French
    'hr': 'hr_core_news_sm',  # Croatian
    'it': 'it_core_news_sm',  # Italian
    'ja': 'ja_core_news_sm',  # Japanese
    'ko': 'ko_core_news_sm',  # Korean
    'lt': 'lt_core_news_sm',  # Lithuanian
    'mk': 'mk_core_news_sm',  # Macedonian
    'nb': 'nb_core_news_sm',  # Norwegian Bokmål
    'nl': 'nl_core_news_sm',  # Dutch
    'pl': 'pl_core_news_sm',  # Polish
    'pt': 'pt_core_news_sm',  # Portuguese
    'ro': 'ro_core_news_sm',  # Romanian
    'sl': 'sl_core_news_sm',  # Slovenian
    'sv': 'sv_core_news_sm',  # Swedish
    'ru': 'ru_core_news_sm',  # Russian
    'uk': 'uk_core_news_sm',  # Ukrainian
    'xx_ent_wiki': 'xx_ent_wiki_sm',  # Multi-language Entity Recognition
    'xx_sent_ud': 'xx_sent_ud_sm',  # Multi-language Sentence Segmentation
    'zh': 'zh_core_web_sm',  # Chinese
}

sm = spacy_models.values()

# Create a dictionary to store NER labels for each model
ner_labels_by_model = {}

# Iterate over the spacy_models
for spacy_model in sm:
    try:
        # Load the model
        nlp = spacy.load(spacy_model)
        
        # Check if the "ner" component exists in the pipeline
        if "ner" in nlp.pipe_names:
            # Get the NER labels for the model
            ner_labels = nlp.get_pipe("ner").labels
            
            # Store the NER labels in the dictionary
            ner_labels_by_model[spacy_model] = ner_labels
        else:
            print(f"Model {spacy_model} does not have NER component.")
    except OSError as e:
        print(f"Error loading model {spacy_model}: {e}")
        # Attempt to download the model
        try:
            spacy.cli.download(spacy_model)
            print(f"Model {spacy_model} downloaded successfully.")
        except Exception as download_error:
            print(f"Error downloading model {spacy_model}: {download_error}")

# Print the NER labels for each model
for model, labels in ner_labels_by_model.items():
    print(f"Model: {model}")
    print(f"NER Labels: {labels}") 



Model xx_sent_ud_sm does not have NER component.
Model: ca_core_news_sm
NER Labels: ('LOC', 'MISC', 'ORG', 'PER')
Model: da_core_news_sm
NER Labels: ('LOC', 'MISC', 'ORG', 'PER')
Model: de_core_news_sm
NER Labels: ('LOC', 'MISC', 'ORG', 'PER')
Model: el_core_news_sm
NER Labels: ('EVENT', 'GPE', 'LOC', 'ORG', 'PERSON', 'PRODUCT')
Model: en_core_web_md
NER Labels: ('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')
Model: es_core_news_sm
NER Labels: ('LOC', 'MISC', 'ORG', 'PER')
Model: fi_core_news_sm
NER Labels: ('CARDINAL', 'DATE', 'EVENT', 'FAC', 'GPE', 'LANGUAGE', 'LAW', 'LOC', 'MONEY', 'NORP', 'ORDINAL', 'ORG', 'PERCENT', 'PERSON', 'PRODUCT', 'QUANTITY', 'TIME', 'WORK_OF_ART')
Model: fr_core_news_sm
NER Labels: ('LOC', 'MISC', 'ORG', 'PER')
Model: hr_core_news_sm
NER Labels: ('DERIV_PER', 'LOC', 'MISC', 'ORG', 'PER')
Model: it_core_news_sm
NER Labels: ('LOC', 'MISC

In [76]:
mappingKeys = set()
for k, v in label_mappings.items():
    mappingKeys.update(v.values())

appendings = {}

for model, labels in ner_labels_by_model.items():
    subdict = {}

    for label in labels:
        exact_match = None

        # Check for an exact match between label and mappingKeys
        if label == 'PER':
            label = 'PERSON'
        
            
        if label in mappingKeys:
            exact_match = label
            subdict[label] = label  # Map it directly
            

        # If no exact match, manually review and select a mapping
        if exact_match is None:
            
            print("Possible mapping keys:")
            print(f"NER label in model '{model}': {label}")
            for index, map in enumerate(mappingKeys):
                print(f"{index + 1}. {map}")

            while True:
                choice = input("Enter the number of the mapping key or 'skip' to skip this label: ")
                if choice == 'skip':
                    break
                try:
                    choice = int(choice)
                    if 1 <= choice <= len(mappingKeys):
                        chosen_mapping = list(mappingKeys)[choice - 1]
                        subdict[label] = chosen_mapping
                        print(f"Mapping '{label}' -> '{chosen_mapping}' for model '{model}'")
                        break
                    else:
                        print("Invalid choice. Please enter a valid number or 'skip'.")
                except ValueError:
                    print("Invalid input. Please enter a valid number or 'skip'.")

    appendings[model] = subdict

print("Mapping results:")
for model, subdict in appendings.items():
    print(f"Model: {model}")
    print(f"Mapping: {subdict}")

    
    


Possible mapping keys:
NER label in model 'hr_core_news_sm': DERIV_PER
1. ORDINAL
2. MISC
3. LAW
4. EVENT
5. FAC
6. QUANTITY
7. NORP
8. TIME
9. CARDINAL
10. PERCENT
11. LANGUAGE
12. DATE
13. MOVEMENT
14. TITLE_AFFIX
15. PERSON
16. PRODUCT
17. ORG
18. WORK_OF_ART
19. PHONE
20. GPE
21. LOC
22. MONEY


Enter the number of the mapping key or 'skip' to skip this label:  15


Mapping 'DERIV_PER' -> 'PERSON' for model 'hr_core_news_sm'
Possible mapping keys:
NER label in model 'ja_core_news_sm': PET_NAME
1. ORDINAL
2. MISC
3. LAW
4. EVENT
5. FAC
6. QUANTITY
7. NORP
8. TIME
9. CARDINAL
10. PERCENT
11. LANGUAGE
12. DATE
13. MOVEMENT
14. TITLE_AFFIX
15. PERSON
16. PRODUCT
17. ORG
18. WORK_OF_ART
19. PHONE
20. GPE
21. LOC
22. MONEY


Enter the number of the mapping key or 'skip' to skip this label:  15


Mapping 'PET_NAME' -> 'PERSON' for model 'ja_core_news_sm'
Possible mapping keys:
NER label in model 'ko_core_news_sm': DT
1. ORDINAL
2. MISC
3. LAW
4. EVENT
5. FAC
6. QUANTITY
7. NORP
8. TIME
9. CARDINAL
10. PERCENT
11. LANGUAGE
12. DATE
13. MOVEMENT
14. TITLE_AFFIX
15. PERSON
16. PRODUCT
17. ORG
18. WORK_OF_ART
19. PHONE
20. GPE
21. LOC
22. MONEY


...

Mapping 'PRS' -> 'PERSON' for model 'sv_core_news_sm'
Possible mapping keys:
NER label in model 'sv_core_news_sm': TME
1. ORDINAL
2. MISC
3. LAW
4. EVENT
5. FAC
6. QUANTITY
7. NORP
8. TIME
9. CARDINAL
10. PERCENT
11. LANGUAGE
12. DATE
13. MOVEMENT
14. TITLE_AFFIX
15. PERSON
16. PRODUCT
17. ORG
18. WORK_OF_ART
19. PHONE
20. GPE
21. LOC
22. MONEY


Enter the number of the mapping key or 'skip' to skip this label:  8


Mapping 'TME' -> 'TIME' for model 'sv_core_news_sm'
Possible mapping keys:
NER label in model 'sv_core_news_sm': WRK
1. ORDINAL
2. MISC
3. LAW
4. EVENT
5. FAC
6. QUANTITY
7. NORP
8. TIME
9. CARDINAL
10. PERCENT
11. LANGUAGE
12. DATE
13. MOVEMENT
14. TITLE_AFFIX
15. PERSON
16. PRODUCT
17. ORG
18. WORK_OF_ART
19. PHONE
20. GPE
21. LOC
22. MONEY


Enter the number of the mapping key or 'skip' to skip this label:  17


Mapping 'WRK' -> 'ORG' for model 'sv_core_news_sm'
Mapping results:
Model: ca_core_news_sm
Mapping: {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}
Model: da_core_news_sm
Mapping: {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}
Model: de_core_news_sm
Mapping: {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}
Model: el_core_news_sm
Mapping: {'EVENT': 'EVENT', 'GPE': 'GPE', 'LOC': 'LOC', 'ORG': 'ORG', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT'}
Model: en_core_web_md
Mapping: {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE', 'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY', 'NORP': 'NORP', 'ORDINAL': 'ORDINAL', 'ORG': 'ORG', 'PERCENT': 'PERCENT', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT', 'QUANTITY': 'QUANTITY', 'TIME': 'TIME', 'WORK_OF_ART': 'WORK_OF_ART'}
Model: es_core_news_sm
Mapping: {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}
Model: fi_core_news_sm
Mapping: {'C

In [77]:
print(appendings)

{'ca_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}, 'da_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}, 'de_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}, 'el_core_news_sm': {'EVENT': 'EVENT', 'GPE': 'GPE', 'LOC': 'LOC', 'ORG': 'ORG', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT'}, 'en_core_web_md': {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE', 'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY', 'NORP': 'NORP', 'ORDINAL': 'ORDINAL', 'ORG': 'ORG', 'PERCENT': 'PERCENT', 'PERSON': 'PERSON', 'PRODUCT': 'PRODUCT', 'QUANTITY': 'QUANTITY', 'TIME': 'TIME', 'WORK_OF_ART': 'WORK_OF_ART'}, 'es_core_news_sm': {'LOC': 'LOC', 'MISC': 'MISC', 'ORG': 'ORG', 'PERSON': 'PERSON'}, 'fi_core_news_sm': {'CARDINAL': 'CARDINAL', 'DATE': 'DATE', 'EVENT': 'EVENT', 'FAC': 'FAC', 'GPE': 'GPE', 'LANGUAGE': 'LANGUAGE', 'LAW': 'LAW', 'LOC': 'LOC', 'MONEY': 'MONEY'

In [78]:

# Dictionary to map NER labels for each model to English labels
ner_label_mapping = {}

# Define label mappings for each model
label_mappings = appendings

# Populate the dictionary with mappings
for model, label_mapping in label_mappings.items():
    ner_label_mapping[model] = label_mapping

new_dict = dict()
for model_labels in label_mappings.values():
    for label in model_labels.values():
        new_dict[label] = []
print(new_dict.keys())

for k,v in label_mappings.items():
    for key,value in v.items():
        for j,z in new_dict.items():
            if value == j:
                z.append(key)
            else:
                pass
for j,z in new_dict.items():
    clean = list(set(z))
    new_dict[j] = clean

others = new_dict['TITLE_AFFIX']+new_dict['PHONE']+new_dict['MISC']
for x in new_dict['TIME']:
    new_dict['DATE'].append(x)
for loc in new_dict['LOC']:
    new_dict['GPE'].append(loc)
numbers = new_dict['CARDINAL']+new_dict['ORDINAL']+new_dict['QUANTITY']+new_dict['PERCENT']
new_dict['OTHER'] = others
new_dict['NUMBER'] = numbers
new_dict.pop('CARDINAL')
new_dict.pop('PERCENT')
new_dict.pop('ORDINAL')
new_dict.pop('QUANTITY')
new_dict.pop('TITLE_AFFIX')
new_dict.pop('PHONE')
new_dict.pop('TIME')
new_dict.pop('LOC')
new_dict.pop('MISC')
for k,v in new_dict.items():
    print(k,'->',v)

dict_keys(['LOC', 'MISC', 'ORG', 'PERSON', 'EVENT', 'GPE', 'PRODUCT', 'CARDINAL', 'DATE', 'FAC', 'LANGUAGE', 'LAW', 'MONEY', 'NORP', 'ORDINAL', 'PERCENT', 'QUANTITY', 'TIME', 'WORK_OF_ART', 'MOVEMENT', 'PHONE', 'TITLE_AFFIX'])
ORG -> ['ORGANIZATION', 'WRK', 'ORG', 'OG', 'MSR', 'orgName']
PERSON -> ['DERIV_PER', 'PERSON', 'PRS', 'persName', 'PET_NAME', 'PS']
EVENT -> ['EVENT', 'EVT', 'EVN']
GPE -> ['GPE', 'LC', 'GPE_ORG', 'geogName', 'GPE_LOC', 'LOC', 'placeName']
PRODUCT -> ['PRODUCT', 'PROD', 'OBJ']
DATE -> ['PERIOD', 'DATE', 'TME', 'TIME', 'time', 'DATETIME', 'TI', 'DT']
FAC -> ['FAC', 'FACILITY']
LANGUAGE -> ['LANGUAGE']
LAW -> ['LAW']
MONEY -> ['MONEY']
NORP -> ['NORP', 'date', 'NAT_REL_POL']
WORK_OF_ART -> ['WORK_OF_ART']
MOVEMENT -> ['MOVEMENT']
OTHER -> ['TITLE_AFFIX', 'PHONE', 'MISC', 'DRV']
NUMBER -> ['CARDINAL', 'NUMERIC_VALUE', 'ORDINAL', 'QT', 'QUANTITY', 'PERCENT']


In [96]:
new_dict

{'ORG': ['ORGANIZATION', 'WRK', 'ORG', 'OG', 'MSR', 'orgName'],
 'PERSON': ['DERIV_PER', 'PERSON', 'PRS', 'persName', 'PET_NAME', 'PS'],
 'EVENT': ['EVENT', 'EVT', 'EVN'],
 'GPE': ['GPE', 'LC', 'GPE_ORG', 'geogName', 'GPE_LOC', 'LOC', 'placeName'],
 'PRODUCT': ['PRODUCT', 'PROD', 'OBJ'],
 'DATE': ['PERIOD', 'DATE', 'TME', 'TIME', 'time', 'DATETIME', 'TI', 'DT'],
 'FAC': ['FAC', 'FACILITY'],
 'LANGUAGE': ['LANGUAGE'],
 'LAW': ['LAW'],
 'MONEY': ['MONEY'],
 'NORP': ['NORP', 'date', 'NAT_REL_POL'],
 'WORK_OF_ART': ['WORK_OF_ART'],
 'MOVEMENT': ['MOVEMENT'],
 'OTHER': ['TITLE_AFFIX', 'PHONE', 'MISC', 'DRV'],
 'NUMBER': ['CARDINAL',
  'NUMERIC_VALUE',
  'ORDINAL',
  'QT',
  'QUANTITY',
  'PERCENT']}

In [79]:
def count_elements(lst):
    element_count = {}
    for item in lst:
        if item in element_count:
            element_count[item] += 1
        else:
            element_count[item] = 1
    return element_count

  
provs = []
for k,v in items:
    
    prov = v['lotProvenance']
    for ke,va in prov.items():
        if va is None:
            pass
        elif type(va) != dict:
            pass
        else:
            provs.append(va)
            
        
provs[0]
spacy_models = []
for x in provs:
    m = x['spacy_model']
    spacy_models.append(m)

models_count = count_elements(spacy_models)
print(models_count)

            

{'zh_core_web_sm': 7402, 'en_core_web_md': 90651, 'de_core_news_sm': 2264, 'it_core_news_sm': 738, 'sv_core_news_sm': 156, 'lt_core_news_sm': 109, 'sl_core_news_sm': 41, 'es_core_news_sm': 536, 'pl_core_news_sm': 435, 'pt_core_news_sm': 110, 'fr_core_news_sm': 4691, 'nl_core_news_sm': 722, 'fi_core_news_sm': 80, 'ro_core_news_sm': 68, 'ca_core_news_sm': 41, 'ja_core_news_sm': 63, 'da_core_news_sm': 227, 'nb_core_news_sm': 14, 'hr_core_news_sm': 3}


In [80]:
NER_eval
new_eval = []
for x in NER_eval:
    for k,v in new_dict.items():
        if x[0] in v:
            new_eval.append((k,x[1],x[2]))

In [81]:
NER_res = []
labels = new_dict.keys()
labels = list(labels)
labels.append('OTHER')
print(len(NER_eval))
exceptions = ['DATE','NUMBER']
for x in new_eval:
    if x[0] in exceptions:
        pass
    else:
            
        # Flag to indicate whether a match is found in NER_res
        match_found = False
        
        for res in NER_res:
            if (x[0], x[1]) == (res[0], res[1]):
                match_found = True
                break  # Exit the loop if a match is found
        
        if not match_found:
            print(x)
            
            while True:
                answer = input("1 if it's OK, add the correct tag if not: ")
                if answer == "1":
                    y = (x[0], x[1], 1, x[0], x[2])
                    break  # Exit the loop when valid input is provided
                elif answer.upper() in labels:
                    y = (x[0], x[1], 0, answer.upper(), x[2])
                    break  # Exit the loop when valid input is provided
                else:
                    print("Invalid input. Please enter 0 or 1 or a valid label.")
            
            NER_res.append(y)

print(NER_res)

2417
('ORG', "Christie's Los Angeles", 'en_core_web_md')


1 if it's OK, add the correct tag if not:  1


('ORG', 'Myth', 'en_core_web_md')


1 if it's OK, add the correct tag if not:  MISC


Invalid input. Please enter 0 or 1 or a valid label.


...

('PERSON', 'Anthony Jacobs', 'en_core_web_md')


1 if it's OK, add the correct tag if not:  1


('PERSON', 'Wilhelm Horn', 'en_core_web_md')


1 if it's OK, add the correct tag if not:  1


[('ORG', "Christie's Los Angeles", 1, 'ORG', 'en_core_web_md'), ('ORG', 'Myth', 0, 'OTHER', 'en_core_web_md'), ('NORP', 'Chinese', 1, 'NORP', 'en_core_web_md'), ('PERSON', 'Julius Eberhardt', 1, 'PERSON', 'en_core_web_md'), ('GPE', 'Nagel Stuttgart', 1, 'GPE', 'en_core_web_md'), ('FAC', '洛杉磯佳士得', 1, 'FAC', 'zh_core_web_sm'), ('WORK_OF_ART', 'Myth and Reality', 1, 'WORK_OF_ART', 'zh_core_web_sm'), ('PERSON', '朱利思∙', 1, 'PERSON', 'zh_core_web_sm'), ('PERSON', '艾伯哈特', 1, 'PERSON', 'zh_core_web_sm'), ('ORG', "Sotheby's Hong Kong", 1, 'ORG', 'en_core_web_md'), ('ORG', "Sotheby's", 1, 'ORG', 'en_core_web_md'), ('GPE', 'Paris', 1, 'GPE', 'en_core_web_md'), ('PERSON', 'Karim Grusenmeyer', 1, 'PERSON', 'en_core_web_md'), ('ORG', "Sotheby's New York", 1, 'ORG', 'en_core_web_md'), ('PERSON', 'Abraham Rosman', 1, 'PERSON', 'en_core_web_md'), ('PERSON', 'Paula Rubel', 1, 'PERSON', 'en_core_web_md'), ('GPE', 'New York', 1, 'GPE', 'en_core_web_md'), ('ORG', "Christie's", 1, 'ORG', 'en_core_web_md'), 

In [82]:
good = 0
bad = 0
bads = []

for x in NER_res:
    if x[2] == 1:
        good += 1
    else:
        bad += 1
        bads.append(x)

print(f'good {good}')
print(f'bad {bad}')

# Count the occurrences of each model in the most inaccurates
from collections import Counter
model_counts = Counter(x[4] for x in bads)

# Find the most inaccurate tags
most_inaccurates_tags = Counter([f"{x[0]} instead of {x[3]}" for x in bads])

print(f"Counts of models in most inaccurates: {model_counts}")
print(f"Most inaccurate tags: {most_inaccurates_tags}")


good 780
bad 273
Counts of models in most inaccurates: Counter({'en_core_web_md': 123, 'fr_core_news_sm': 52, 'de_core_news_sm': 20, 'nl_core_news_sm': 19, 'it_core_news_sm': 18, 'zh_core_web_sm': 18, 'es_core_news_sm': 9, 'pt_core_news_sm': 5, 'pl_core_news_sm': 3, 'fi_core_news_sm': 2, 'lt_core_news_sm': 2, 'da_core_news_sm': 1, 'nb_core_news_sm': 1})
Most inaccurate tags: Counter({'ORG instead of OTHER': 27, 'OTHER instead of DATE': 27, 'PERSON instead of ORG': 23, 'ORG instead of GPE': 22, 'GPE instead of OTHER': 20, 'GPE instead of ORG': 20, 'ORG instead of PERSON': 15, 'PERSON instead of OTHER': 14, 'OTHER instead of ORG': 13, 'PERSON instead of GPE': 13, 'FAC instead of ORG': 12, 'GPE instead of PERSON': 11, 'FAC instead of OTHER': 5, 'OTHER instead of OTHER': 5, 'WORK_OF_ART instead of OTHER': 4, 'LAW instead of OTHER': 4, 'GPE instead of DATE': 4, 'ORG instead of DATE': 3, 'FAC instead of GPE': 3, 'PRODUCT instead of OTHER': 2, 'OTHER instead of PERSON': 2, 'ORG instead of NUM

In [115]:
TAGS_ = []
for x in bads:
    TAGS_.append(x[3])
tags_ = list(set(TAGS_))

In [116]:
tags_

['GPE', 'NUMBER', 'NORP', 'OTHER', 'PERSON', 'ORG', 'DATE']

In [117]:
bads

[('ORG', 'Myth', 0, 'OTHER', 'en_core_web_md'),
 ('OTHER', 'December 1982', 0, 'DATE', 'fr_core_news_sm'),
 ('OTHER', 'December 1992', 0, 'DATE', 'fr_core_news_sm'),
 ('ORG', 'PRINCELY', 0, 'OTHER', 'en_core_web_md'),
 ('FAC', 'B004', 0, 'OTHER', 'nl_core_news_sm'),
 ('GPE', 'n°', 0, 'OTHER', 'en_core_web_md'),
 ('LANGUAGE', 'Filipino', 0, 'PERSON', 'en_core_web_md'),
 ('GPE', 'Petit Musée', 0, 'ORG', 'fr_core_news_sm'),
 ('PRODUCT', 'Cinq-Mars', 0, 'OTHER', 'en_core_web_md'),
 ('OTHER', 'Sui Yuan', 0, 'PERSON', 'fr_core_news_sm'),
 ('GPE', 'Arte Primitivo', 0, 'ORG', 'en_core_web_md'),
 ('ORG', 'Martin', 0, 'PERSON', 'en_core_web_md'),
 ('ORG', 'Queens', 0, 'GPE', 'fi_core_news_sm'),
 ('ORG', 'MIAMI', 0, 'GPE', 'en_core_web_md'),
 ('ORG', 'XII', 0, 'NUMBER', 'en_core_web_md'),
 ('OTHER', 'Galerie Démons', 0, 'ORG', 'fr_core_news_sm'),
 ('GPE', 'Merveilles', 0, 'OTHER', 'fr_core_news_sm'),
 ('PERSON', 'Lowther Castle', 0, 'ORG', 'en_core_web_md'),
 ('FAC', 'the Tuyet Nguyet', 0, 'ORG',

In [94]:
tags_res = []
for x in NER_res:
    if x[3] in tags_res:
        pass
    else:
        tags_res.append(x[3])
print(tags_res)

dict_res = {}
for x in tags_res:
    dict_res[x] = []
    for y in NER_res:
        if y[3] == x:
            dict_res[x].append(y[1])


['ORG', 'OTHER', 'NORP', 'PERSON', 'GPE', 'FAC', 'WORK_OF_ART', 'DATE', 'NUMBER', 'MONEY']


In [95]:
dict_res

{'ORG': ["Christie's Los Angeles",
  "Sotheby's Hong Kong",
  "Sotheby's",
  "Sotheby's New York",
  "Christie's",
  'Private Collection',
  'American Private Collection',
  'Leonard Hutton Galleries',
  'Collection Mrs',
  'European Private Collection',
  "Sotheby's London",
  "Christie's East New York",
  'Kerr Collection',
  "D'Arcy Galleries",
  'the Museum of Fine Arts',
  'Petit Musée',
  'S. Marchant & Son',
  "Christie's London",
  'Arte Primitivo',
  'Royal-Athena Galleries',
  'Galerie Démons',
  'Charles Ede Ltd',
  'Lowther Castle',
  'London Market',
  'Florida Private Collection',
  'the Tuyet Nguyet',
  'Earl of Sandwich',
  "Christie'",
  'Barakat Antiquities',
  'Corte Madera',
  'Butterfield & Butterfield',
  'GENTLEMAN',
  'Christie’s',
  'KILLADOON HOUSE',
  'CRG Gallery',
  'Clarks Auction',
  "Christie's Hong Kong",
  'the Brooklyn Museum',
  'Hotspur Ltd',
  'Mallett & Son Ltd.',
  'Dominique Thirion Gallery',
  'Collection Particulière',
  'New York Private Coll

In [114]:

with open(file, 'r') as input_f:

    # Returns json object as a dictionary
    data = json.load(input_f)

    # Iterating through the json list
    # Iterating through the json list
    for k, v in tqdm(data['lots'].items()):

        json_object = v
        prov = v['lotProvenance']
        events_of_artworks = []

        # Consider only artworks with provenance
        if (prov):
            try:
                # Preprocessing
                for k, v in prov.items():
                    if v is None:
                        pass
                    else:
                        prov = clean_provenance(v)

                        # Remove dots from names, etc.
                        entities = extract_named_entities(ner_model, prov)
                        for i, entity_ls in enumerate(entities.values()):
                            print(i,entity_ls)
                            break
                                

SyntaxError: unexpected EOF while parsing (4129644338.py, line 29)

In [ ]:
'''
                            break
                            for i, entity_ls in enumerate(entities.values()):
                                if (list(entities.keys())[i] != 'DATE'):
                                    for e in entity_ls:
                                        prov = prov.replace(e, utils.remove_dots(e))

                            events = extract_events_dot(prov)

                            # Extract named entities from the events
                            artwork_events = []

                            for event in events:
                                ev_data = {}
                                ev_data['label'] = event

                                ev_entites = utils.extract_named_entities(ner_model, event)

                                for entity_type in ev_entites.keys():
                                    ev_data[entity_type] = ev_entites[entity_type]

                                artwork_events.append(ev_data)
                                print('oLDone\n', artwork_events)
                                from collections import Counter
                                new_artwork_events = []
                                from collections import Counter

                                for event in artwork_events:
                                    new_elems = []
                                    label_value = None  # Initialize a variable to store the 'label' value
                                    for k, v in event.items():
                                        if k == 'label':  # Check if the key is 'label'
                                            label_value = v
                                            new_elems.append((k, v))
                                        if type(v) is str:
                                            if v in annosk:
                                                idx = annosk.index(v)
                                                new_elems.append((annosv[idx], v))
                                                print('is str in annosk', new_elems)
                                            elif 'collection' in v.lower():
                                                if k == 'label':
                                                    if len(event) == 1:
                                                        new_elems.append(('ORG', v))
                                                        print('collection in v lower with label k', new_elems)
                                                        new_elems.append(('label', v))
                                                        print('readding label', new_elems)
                                                else:
                                                    new_elems.append(('ORG', v))
                                                    print('not label k', new_elems)
                                            else:
                                                new_elems.append((k, v))
                                                print('else', new_elems)
                                        elif type(v) is list:
                                            key = k
                                            for element in v:
                                                if element in annosk:
                                                    idx = annosk.index(element)
                                                    new_elems.append((annosv[idx], element))
                                                    print('list', new_elems)
                                                elif 'collection' in element.lower():
                                                    if key == 'label':
                                                        if len(event) == 1:
                                                            new_elems.append(('ORG', element))
                                                            print('listwithcollectionlabel', new_elems)
                                                            new_elems.append(('label', element))
                                                            print('readdinglabel', new_elems)
                                                    else:
                                                        new_elems.append(('ORG', element))
                                                        print('notlist', new_elems)
                                                else:
                                                    new_elems.append((k, element))
                                                    print(new_elems)
                                    event['new_elem'] = new_elems
                                    output_dict = {}

                                    for elem in event['new_elem']:
                                        key, value = elem

                                        if key in output_dict:
                                            if isinstance(output_dict[key], list):
                                                output_dict[key].append(value)
                                            else:
                                                output_dict[key] = [output_dict[key], value]
                                        else:
                                            output_dict[key] = value

                                    output_dict['label'] = label_value  # Restore the 'label' key-value pair
                                    event.clear()
                                    event.update(output_dict)

                                print('newone\n', artwork_events)

                        events_of_artworks.append(artwork_events)

                    with open('events/events.txt', 'a', encoding="utf-8") as output_f:
                        json_object["events"] = events_of_artworks
                        json_str = json.dumps({str(artwork_index): json_object}, ensure_ascii=False)
                        output_f.write(json_str + "\n")
                        artwork_index += 1
                except:
                    with open('events/events.txt', 'a', encoding="utf-8") as output_f:
                        json_object["events"] = ""
                        json_str = json.dumps({str(artwork_index): json_object}, ensure_ascii=False)
                        output_f.write(json_str + "\n")
                        artwork_index += 1
                    with open('events/noevents.txt', 'a', encoding="utf-8") as output_f:
                        json_object["eventy"] = ""
                        json_str = json.dumps({str(artwork_index): json_object}, ensure_ascii=False)
                        output_f.write(json_str + "\n")
                        artwork_index += 1
            else:
                with open('events/events.txt', 'a', encoding="utf-8") as output_f:
                    json_object["events"] = ""
                    json_str = json.dumps({str(artwork_index): json_object}, ensure_ascii=False)
                    output_f.write(json_str + "\n")
                    artwork_index += 1
                with open('events/noevents.txt', 'a', encoding="utf-8") as output_f:
                    json_object["events"] = ""
                    json_str = json.dumps({str(artwork_index): json_object}, ensure_ascii=False)
                    output_f.write(json_str + "\n")
                    artwork_index += 1

    return artwork_index'''